In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

import face_detect as FD
import insightface_detect as ID

# face_recognition

In [23]:
import cv2
from IPython.display import display, Image

face_cascade = FD.initialize()
video = cv2.VideoCapture('http://192.168.2.36:4747/video')
video.set(3,480) # adjust width
video.set(4,480) # adjust height
video.set(cv2.CAP_PROP_BUFFERSIZE, 0)
display_handle=display(None, display_id=True)
try:
    while True:
        _, frame = video.read()
        result = FD.face_detect(frame, face_cascade)
        _, result = cv2.imencode('.jpeg', result)
        display_handle.update(Image(data=result.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    display_handle.update(None)

None

# insightface

In [24]:
import cv2
from IPython.display import display, Image

face_cascade = ID.initialize()
video = cv2.VideoCapture('http://192.168.2.36:4747/video')
video.set(3,480) # adjust width
video.set(4,480) # adjust height
video.set(cv2.CAP_PROP_BUFFERSIZE, 10)
display_handle=display(None, display_id=True)
try:
    while True:
        _, frame = video.read()
        result = ID.person_detect(frame, face_cascade)
        _, result = cv2.imencode('.jpeg', result)
        display_handle.update(Image(data=result.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    display_handle.update(None)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


None

# deepface

In [59]:
from deepface import DeepFace
from deepface.detectors import FaceDetector
from deepface.commons import functions


def detect_face(img, backend, align = True):

    #detector stored in a global variable in FaceDetector object.
    #this call should be completed very fast because it will return found in memory
    #it will not build face detector model in each call (consider for loops)
    face_detector = FaceDetector.build_model(backend)
    detected_face = None
    img_region = None
    try:
        detected_face, img_region = FaceDetector.detect_face(face_detector, backend, img, align)
    except:
        pass
    return detected_face, img_region

# def detect_face(detector, wrapper, img, align = True):

#     faces = wrapper(detector, img, align)
#     return faces

# def resize_image(img, target_size=(224,224), grayscale=False):
#     factor_0 = target_size[0] / img.shape[0]
#     factor_1 = target_size[1] / img.shape[1]
#     factor = min(factor_0, factor_1)

#     dsize = (int(img.shape[1] * factor), int(img.shape[0] * factor))
#     img = cv2.resize(img, dsize)

#     # Then pad the other side to the target size by adding black pixels
#     diff_0 = target_size[0] - img.shape[0]
#     diff_1 = target_size[1] - img.shape[1]
#     if grayscale == False:
#         # Put the base image in the middle of the padded image
#         img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)), 'constant')
#     else:
#         img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2)), 'constant')
        
#     if img.shape[0:2] != target_size:
#         img = cv2.resize(img, target_size)
        
#     return img

# def normalize(image):
#     img_pixels = np.expand_dims(image, axis = 0)
#     img_pixels /= 255 #normalize input in [0, 1]
#     return img_pixels

# def preprocess_face(face_img, target_size=(224, 224)):
#     image = resize_image(face_img, target_size)
#     norm_image = normalize(image)
#     return norm_image



recognizer = DeepFace.build_model('ArcFace')
detector_backends = [
    'opencv', 
    'ssd', 
    'dlib', 
    'mtcnn', 
    'retinaface', 
    'mediapipe'
]
be = detector_backends[1]

In [88]:
%%time

for _ in range(10):
    recognizer = DeepFace.build_model('ArcFace')

CPU times: user 2.3 ms, sys: 342 µs, total: 2.65 ms
Wall time: 2.29 ms


In [90]:
%%timeit

recognizer = DeepFace.build_model('ArcFace')

844 ns ± 4.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [74]:
def preprocess_face_image(img, target_size = (224,224,), grayscale=False):
    factor_0 = target_size[0] / img.shape[0]
    factor_1 = target_size[1] / img.shape[1]
    factor = min(factor_0, factor_1)

    dsize = (int(img.shape[1] * factor), int(img.shape[0] * factor))
    img = cv2.resize(img, dsize)

    # Then pad the other side to the target size by adding black pixels
    diff_0 = target_size[0] - img.shape[0]
    diff_1 = target_size[1] - img.shape[1]
    if grayscale == False:
        # Put the base image in the middle of the padded image
        img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)), 'constant')
    else:
        img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2)), 'constant')

    if img.shape[0:2] != target_size:
        img = cv2.resize(img, target_size)

    img_pixels = np.expand_dims(img, axis = 0).astype(np.float32)
    img_pixels /= 255 #normalize input in [0, 1]
    
    return img_pixels

In [80]:
import cv2
from IPython.display import display, Image, JSON

video = cv2.VideoCapture('http://192.168.2.36:4747/video')
video.set(3,640) # adjust width
video.set(4,640) # adjust height
video.set(cv2.CAP_PROP_BUFFERSIZE, 0)
display_handle=display(None, display_id=True)
try:
    while True:
        _, frame = video.read()
        face, location = detect_face(frame, be, align=False)
        
        embedding = None
        result = frame.copy()
        if face is not None and len(face) != 0:
            result = face.copy()
            img = preprocess_face_image(face, target_size=(112,112))
            img = functions.normalize_input(img = img, normalization = 'base')
            embedding = recognizer(img).numpy()[0].tolist()
        
        _, result = cv2.imencode('.jpeg', result)
        
#         display_handle.update(JSON(data=embedding))
        display_handle.update(Image(data=result.tobytes()))
        
except KeyboardInterrupt:
    pass
finally:
    video.release()
    display_handle.update(None)

None

In [83]:
img = preprocess_face_image(face, target_size=(112,112))
img = functions.normalize_input(img = img, normalization = 'base')
embedding = recognizer(img).numpy()[0].tolist()

In [84]:
len(embedding)

512

In [ ]:
from deepface.commons import functions


input_shape_x, input_shape_y = functions.find_input_shape(model)

	#detect and align
	img = functions.preprocess_face(img = img_path
		, target_size=(input_shape_y, input_shape_x)
		, enforce_detection = enforce_detection
		, detector_backend = detector_backend
		, align = align)

	#---------------------------------
	#custom normalization

	img = functions.normalize_input(img = img, normalization = normalization)